In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable 

import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_raw = pd.read_csv(r'/kaggle/input/sign-language-mnist/sign_mnist_train.csv', sep=',')
test_raw = pd.read_csv(r'/kaggle/input/sign-language-mnist/sign_mnist_test.csv', sep=',')

In [ ]:
train_label = train_raw.pop('label')
train = train_raw.values
train_label = train_label.values
test_label = test_raw.pop('label')
test = test_raw.values
test_label = test_label.values

In [ ]:
fig = plt.figure(figsize=(10,8))
columns=4
rows=4
for i in range(16):
    ax = plt.subplot(4,4,i+1)
    plt.imshow(train[i].reshape(28,28))

In [ ]:
def preprocessing(data, label):
    data_list=[]
    for i in data:
        data_list.append(i.reshape((1,28,28)))
    data = torch.FloatTensor(np.array(data_list))
    label = torch.LongTensor(label.tolist())
    return data, label

In [ ]:
train, train_label = preprocessing(train, train_label)
test, test_label = preprocessing(test, test_label)

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.conv1 = nn.Conv2d(1,32,kernel_size=(3,3), padding=(1,1), stride=(1,1))
        self.conv2 = nn.Conv2d(32,32,kernel_size=(3,3), padding=(1,1), stride=(1,1))
        self.conv3 = nn.Conv2d(32,64,kernel_size=(3,3), padding=(1,1), stride=(1,1))
        self.adapt = nn.AdaptiveMaxPool2d((3,3))
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(64*3*3, 576)
        self.fc2 = nn.Linear(576, 26)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.adapt(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return self.softmax(x)
    def evaluation(self, result, label):
        num=0
        for x, y in zip(result, label):
            if x==y:
                num+=1
        acc = num/len(result)
        return acc

In [ ]:
print(Classifier())

In [ ]:
epochs=50
batch_size=256
model = Classifier()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.3)
loss_function = nn.CrossEntropyLoss()
loss_log = []
acc_log = []
for epoch in range(epochs):
    for i in range(0, train.shape[0], batch_size):
        batch_x = train[i: i+batch_size]
        batch_y = train_label[i: i+batch_size]
        optimizer.zero_grad()
        output = model(Variable(batch_x))
        loss = loss_function(output, Variable(batch_y))
        loss.backward()
        optimizer.step()
    acc_log.append(model.evaluation(torch.max(model(Variable(test)).data, 1)[1], test_label))
    loss_log.append(loss.item())  
    print('Epoch: {} - Loss: {:.6f}'.format(epoch + 1, loss.item()))

In [ ]:
plt.figure(figsize = (13,5))                

for i in range(2):
    ax = plt.subplot(1,2,i+1)
    if i==0:
        plt.plot(loss_log, label='Loss', color='orange')
    plt.legend(frameon=False)
    if i==1:
        plt.plot(acc_log, label='Accuracy')
    plt.legend(frameon=False)

In [ ]:
acc_log